In [ ]:
# Imports and config
import sys
import os

# For Jupyter notebooks
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root)

import torch
torch.backends.cudnn.benchmark = True

from src.data_loader import get_data_loaders
from src.train import train_model
from src.models.squeezenet import SqueezeNet, SqueezeNetBypass
from src.models.mobilenet import MobileNetV1

# Configuration
DATA_DIR = "../data/FER2013"
BATCH_SIZE = 64
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_EPOCHS = 40  # change to 5-10 for quick runs
LR = 1e-3

print("Device:", DEVICE)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Load data
print("\nLoading data...")
train_loader, test_loader = get_data_loaders(DATA_DIR, batch_size=BATCH_SIZE)
print(f"Training samples: {len(train_loader.dataset)}")
print(f"Testing samples: {len(test_loader.dataset)}")

Device: cuda
PyTorch version: 2.5.1+cu121
CUDA available: True

Loading data...
Training samples: 28709
Testing samples: 7178


In [5]:
# Prepare model save folders
os.makedirs("../models/mobilenet", exist_ok=True)
os.makedirs("../models/squeezenet", exist_ok=True)
os.makedirs("../models/shiftnet", exist_ok=True)

In [6]:
# Train SqueezeNet from scratch
print("\nInitializing SqueezeNet model...")
squeezenet = SqueezeNet(num_classes=7, in_channels=1)

# Print model summary (optional)
total_params = sum(p.numel() for p in squeezenet.parameters())
trainable_params = sum(p.numel() for p in squeezenet.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Train the model
print("\nStarting training...")
squeezenet_trained = train_model(
    squeezenet, 
    train_loader, 
    test_loader, 
    device=DEVICE,
    num_epochs=NUM_EPOCHS, 
    lr=LR
)
# Model is automatically saved to models/<classname>/best_model.pth by train_model

print("\nTraining complete!")


Initializing SqueezeNet model...
Total parameters: 735,559
Trainable parameters: 735,559

Starting training...

Starting Training: SqueezeNet


Epoch [1/40]


Validation: 100%|████████████████████████| 113/113 [00:32<00:00,  3.51it/s, loss=1.9371, acc=27.67%]



────────────────────────────────────────────────────────────
  Train Loss: 1.9782 | Train Acc: 22.32%
  Val Loss:   1.8080 | Val Acc:   27.67%
────────────────────────────────────────────────────────────
  ✓ Best model saved! Accuracy: 27.67%

Epoch [2/40]


Validation: 100%|████████████████████████| 113/113 [00:08<00:00, 13.60it/s, loss=1.0350, acc=35.72%]



────────────────────────────────────────────────────────────
  Train Loss: 1.7676 | Train Acc: 29.43%
  Val Loss:   1.6251 | Val Acc:   35.72%
────────────────────────────────────────────────────────────
  ✓ Best model saved! Accuracy: 35.72%

Epoch [3/40]


...


────────────────────────────────────────────────────────────
  Train Loss: 0.8905 | Train Acc: 66.80%
  Val Loss:   1.0342 | Val Acc:   62.62%
────────────────────────────────────────────────────────────
  ✓ Best model saved! Accuracy: 62.62%

Epoch [39/40]


Validation: 100%|████████████████████████| 113/113 [00:08<00:00, 13.58it/s, loss=0.2964, acc=61.28%]



────────────────────────────────────────────────────────────
  Train Loss: 0.8809 | Train Acc: 67.13%
  Val Loss:   1.0578 | Val Acc:   61.28%
────────────────────────────────────────────────────────────

Epoch [40/40]


Validation: 100%|████████████████████████| 113/113 [00:08<00:00, 13.69it/s, loss=0.1584, acc=61.54%]


────────────────────────────────────────────────────────────
  Train Loss: 0.8726 | Train Acc: 67.40%
  Val Loss:   1.0580 | Val Acc:   61.54%
────────────────────────────────────────────────────────────

Training Completed!
Best Validation Accuracy: 62.62%
Model saved at: models/squeezenet/best_model.pth


Training complete!
